In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

ROOT_FOLDER = "NYPL-menus"

MENU = 1
MENU_PAGE = 3
MENU_ITEM = 2
DISH = 0

In [2]:
dataset = []

for filename in sorted(Path(ROOT_FOLDER).iterdir()):
    if filename.name.endswith(".csv"):
        dataset.append(pd.read_csv(filename, na_values=[""]))

# dataset[MENU].head()
# dataset[MENU_PAGE].head()
# dataset[MENU_ITEM].head()
# dataset[DISH].head()

In [ ]:
# IC 1: Temporal consistency in Dish

In [3]:
# IC 2: Date Outliner in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: call_prefix == date_year when both exist
ic2_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() & 
    dataset[MENU]["date"].notna() & 
    dataset[MENU]["call_prefix"].str.isnumeric() &
    dataset[MENU]["date"].str.match(r"[^1]+") & 
    (dataset[MENU]["call_prefix"] != dataset[MENU]["date_prefix"])
]

print(f"Before Cleaning Applied: {len(ic2_violations)}")
dataset[MENU].loc[ic2_violations.index][["id", "call_number", "date", "call_prefix", "date_prefix"]]

Before Cleaning Applied: 4


,id,call_number,date,call_prefix,date_prefix
525,13112,1900-2328,0190-03-06,1900,0190
14659,32265,1918-0387_wotm,2928-03-26,1918,2928
16917,34727,1912-0667_wotm,0001-01-01,1912,0001
16918,34728,1912-0668_wotm,0001-01-01,1912,0001


In [4]:

# IC 2: Date Outliner in Menu Cleaning
dataset[MENU].loc[ic2_violations.index, "date"] = (
    dataset[MENU].loc[ic2_violations.index, "call_prefix"] + 
    dataset[MENU].loc[ic2_violations.index, "date"].str[4:]
)

print(f"After Cleaning Applied: {len(ic2_violations)}")
dataset[MENU].loc[ic2_violations.index][["id", "call_number", "date", "call_prefix", "date_prefix"]]

After Cleaning Applied: 4


,id,call_number,date,call_prefix,date_prefix
525,13112,1900-2328,1900-03-06,1900,0190
14659,32265,1918-0387_wotm,1918-03-26,1918,2928
16917,34727,1912-0667_wotm,1912-01-01,1912,0001
16918,34728,1912-0668_wotm,1912-01-01,1912,0001
